In [74]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ner-data/Clinical_Dataset.xlsx


In [ ]:
import pandas as pd
import ast
import json
from sklearn.preprocessing import LabelEncoder
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
import torch
from transformers import AutoModel, AutoTokenizer

In [4]:
df=pd.read_excel("/kaggle/input/ner-data/Clinical_Dataset.xlsx")

In [81]:
df.head(2)

,Discharge Summary,entities,sdoh_response,sdoh_proceessed,processed_column,extracted_data,summary,name,extracted_entities,first_entity_token_ids,first_entity_attention_mask,second_entity_token_ids,second_entity_attention_mask
0,\n139391631\nGH\n97368061\n7/20/1999 12:00:00 ...,"b'[[""1"",""NEUROPATHIC PAIN"",""PROBLEM"",0.7168472...","```json\n{\n ""age"": ""65"",\n ""gender"": ""f...","{'age': '65', 'gender': 'female', 'race': '', ...","[[""1"",""NEUROPATHIC PAIN"",""PROBLEM"",0.716847240...",Error parsing data: malformed node or string o...,"[{'start': 45, 'end': 61, 'name': 'neuropathic...","neuropathic pain, thoracotomy syndrome, spinal...","[[NEUROPATHIC PAIN, PROBLEM], [thoracotomy syn...","[[101, 24928, 11955, 21745, 2489, 102], [101, ...","[[1, 1, 1, 1, 1, 1], [1, 1, 1, 0, 0, 0]]","[[101, 24438, 6533, 18982, 18574, 9318, 102], ...","[[1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 0, 0, 0, 0]]"
1,004668411\nCTMC\n68299235\n763052\n9/29/1993 1...,"b'[[""1"",""HIV positive"",""PROBLEM"",0.40534040331...","```json\n{\n ""age"": ""28"",\n ""gender"": ""w...","{'age': '28', 'gender': 'woman', 'race': '', '...","[[""1"",""HIV positive"",""PROBLEM"",0.4053404033184...",Error parsing data: malformed node or string o...,"[{'start': 232, 'end': 244, 'name': 'hiv posit...","hiv positive, left upper quadrant, pain, nause...","[[HIV positive, PROBLEM], [left upper quadrant...","[[101, 20844, 1964, 3112, 102], [101, 2463, 10...","[[1, 1, 1, 1, 1], [1, 1, 1, 0, 0]]","[[101, 1286, 3105, 186, 18413, 6922, 102, 0], ...","[[1, 1, 1, 1, 1, 1, 1, 0], [1, 1, 1, 1, 1, 1, ..."


In [ ]:
df['entities'][0]

In [5]:

# Sample DataFrame with a column 'text_column' containing strings


# Function to remove the first two and last character of a string
def trim_string(s):
    if len(s) > 3:  # Ensure the string is long enough to remove characters
        return s[2:-1]
    else:
        return ''  # Return empty if the string is too short

# Apply the function to each row in the 'text_column'
df['processed_column'] = df['entities'].apply(trim_string)


In [75]:
df.head()

,Discharge Summary,entities,sdoh_response,sdoh_proceessed,processed_column,extracted_data,summary,name,extracted_entities,first_entity_token_ids,first_entity_attention_mask,second_entity_token_ids,second_entity_attention_mask
0,\n139391631\nGH\n97368061\n7/20/1999 12:00:00 ...,"b'[[""1"",""NEUROPATHIC PAIN"",""PROBLEM"",0.7168472...","```json\n{\n ""age"": ""65"",\n ""gender"": ""f...","{'age': '65', 'gender': 'female', 'race': '', ...","[[""1"",""NEUROPATHIC PAIN"",""PROBLEM"",0.716847240...",Error parsing data: malformed node or string o...,"[{'start': 45, 'end': 61, 'name': 'neuropathic...","neuropathic pain, thoracotomy syndrome, spinal...","[[NEUROPATHIC PAIN, PROBLEM], [thoracotomy syn...","[[101, 24928, 11955, 21745, 2489, 102], [101, ...","[[1, 1, 1, 1, 1, 1], [1, 1, 1, 0, 0, 0]]","[[101, 24438, 6533, 18982, 18574, 9318, 102], ...","[[1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 0, 0, 0, 0]]"
1,004668411\nCTMC\n68299235\n763052\n9/29/1993 1...,"b'[[""1"",""HIV positive"",""PROBLEM"",0.40534040331...","```json\n{\n ""age"": ""28"",\n ""gender"": ""w...","{'age': '28', 'gender': 'woman', 'race': '', '...","[[""1"",""HIV positive"",""PROBLEM"",0.4053404033184...",Error parsing data: malformed node or string o...,"[{'start': 232, 'end': 244, 'name': 'hiv posit...","hiv positive, left upper quadrant, pain, nause...","[[HIV positive, PROBLEM], [left upper quadrant...","[[101, 20844, 1964, 3112, 102], [101, 2463, 10...","[[1, 1, 1, 1, 1], [1, 1, 1, 0, 0]]","[[101, 1286, 3105, 186, 18413, 6922, 102, 0], ...","[[1, 1, 1, 1, 1, 1, 1, 0], [1, 1, 1, 1, 1, 1, ..."
2,006544894 \nNVH \n65104826 \n1/2/2004 12:00:00...,"b'[[""1"",""VT"",""PROBLEM"",0.534027636051178,48,50...","```json\n{\n ""age"": ""78"",\n ""gender"": ""M...","{'age': '78', 'gender': 'M', 'race': '', 'educ...","[[""1"",""VT"",""PROBLEM"",0.534027636051178,48,50,[...",Error parsing data: malformed node or string o...,"[{'start': 48, 'end': 50, 'name': 'vt', 'categ...","vt, cardiac cath, stent, amp, annetiff, amioda...","[[VT, PROBLEM], [cardiac cath, PROCEDURE], [st...","[[101, 191, 1204, 102], [101, 2463, 102, 0]]","[[1, 1, 1, 1], [1, 1, 1, 0]]","[[101, 17688, 5855, 1324, 102], [101, 7791, 10...","[[1, 1, 1, 1, 1], [1, 1, 1, 0, 0]]"
3,013496896\nFIH\n4925669\n21983/7jk2\n129165\n0...,"b'[[""1"",""CORONARY ARTERY"",""ANATOMICAL_STRUCTUR...","```json\n{\n ""age"": ""72"",\n ""gender"": ""f...","{'age': '72', 'gender': 'female', 'race': '', ...","[[""1"",""CORONARY ARTERY"",""ANATOMICAL_STRUCTURE""...",Error parsing data: malformed node or string o...,"[{'start': 68, 'end': 83, 'name': 'coronary ar...","coronary artery, stenosis, coronary artery, st...","[[CORONARY ARTERY, ANATOMICAL_STRUCTURE], [STE...","[[101, 1884, 15789, 1616, 18593, 102, 0, 0], [...","[[1, 1, 1, 1, 1, 1, 0, 0], [1, 1, 1, 1, 1, 1, ...","[[101, 188, 5208, 11776, 102], [101, 2463, 102...","[[1, 1, 1, 1, 1], [1, 1, 1, 0, 0]]"
4,014894512\nFIH\n0589930\n86515/6878\n801232\n1...,"b'[[""1"",""OSTEOCHONDROMATOSIS"",""PROBLEM"",0.9136...","```json\n{\n ""age"": ""68"",\n ""gender"": ""m...","{'age': '68', 'gender': 'male', 'race': 'white...","[[""1"",""OSTEOCHONDROMATOSIS"",""PROBLEM"",0.913655...",Error parsing data: malformed node or string o...,"[{'start': 63, 'end': 82, 'name': 'osteochondr...","osteochondromatosis, knee, osteochondromatosis...","[[OSTEOCHONDROMATOSIS, PROBLEM], [KNEE, ANATOM...","[[101, 184, 13894, 9962, 16838, 18885, 26156, ...","[[1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 0, 0, 0, ...","[[101, 5656, 102, 0, 0, 0, 0, 0], [101, 1126, ...","[[1, 1, 1, 0, 0, 0, 0, 0], [1, 1, 1, 1, 1, 1, ..."


In [13]:
df["entities"][0]

'b\'[["1","NEUROPATHIC PAIN","PROBLEM",0.7168472409248352,45,61,["LIKELY",0.9972301125526428],["CURRENT",0.9983813762664795],["PATIENT",0.9996469616889954]],["2","thoracotomy syndrome","PROBLEM",0.6461207270622253,179,199,["LIKELY",0.9994791150093079],["CURRENT",0.9478739500045776],["PATIENT",0.9996469616889954]],["3","spinal cord","ANATOMICAL_STRUCTURE",0.6103355884552002,242,253,null,null,null],["4","stimulator trial","PROCEDURE",0.3881048560142517,254,270,["LIKELY",0.9858991503715515],["CURRENT",0.7197356820106506],["PATIENT",0.9990343451499939]],["5","epidural","PROCEDURE",0.7892853617668152,304,312,["LIKELY",0.9873924851417542],["CURRENT",0.6060205698013306],["PATIENT",0.9974652528762817]],["6","thoracotomy syndrome","PROBLEM",0.7735494375228882,408,428,["LIKELY",0.9992082715034485],["CLINICAL_HISTORY",0.9301432967185974],["PATIENT",0.9997164607048035]],["7","thoracotomy incision","PROCEDURE",0.8911129832267761,462,482,["LIKELY",0.99959796667099],["CLINICAL_HISTORY",0.866425514221

In [17]:
import ast

# Example string data
data = df["entities"][0]

if data.startswith("b'") or data.startswith('b"'):
    clean_data = data[2:-1]  # Remove the leading `b'` or `b"` and trailing `'` or `"`

# Safely evaluate the cleaned string
try:
    list_of_lists = ast.literal_eval(clean_data)
    print(list_of_lists)  # Output: [['aman', 23], ['ram', 12]]
    print(type(list_of_lists))  # Output: <class 'list'>
except Exception as e:
    print("Error evaluating the data:", e)

Error evaluating the data: malformed node or string on line 1: <ast.Name object at 0x7d95d58de3e0>


In [ ]:
import json

# Example string data
# data = '[["1","NEUROPATHIC PAIN","PROBLEM",0.7168472409248352,45,61,["LIKELY",0.9972301125526428],["CURRENT",0.9983813762664795],["PATIENT",0.9996469616889954]]]'

# Parse the string using JSON
try:
    list_of_lists = json.loads(clean_data)
    print(list_of_lists)  # Output: [['1', 'NEUROPATHIC PAIN', 'PROBLEM', 0.7168472409248352, 45, 61, ['LIKELY', 0.9972301125526428], ['CURRENT', 0.9983813762664795], ['PATIENT', 0.9996469616889954]]]
except Exception as e:
    print("Error parsing JSON:", e)


In [23]:
extracted_entities = [[item[1], item[2]] for item in list_of_lists]


In [ ]:


# Assuming df is your DataFrame
# df = pd.DataFrame(...)

# Function to process each row and extract entities
def extract_entities(row):
    data = row["entities"]
    
    # Check if the data needs cleaning (i.e., remove leading 'b' and trailing quotes)
    if isinstance(data, str) and (data.startswith("b'") or data.startswith('b"')):
        clean_data = data[2:-1]  # Remove the leading `b'` or `b"` and trailing `'` or `"`
    else:
        clean_data = data
    
    try:
        # Parse the cleaned string using JSON (or you can use ast.literal_eval if it's a valid Python literal)
        list_of_lists = json.loads(clean_data)

        # Extract the second and third entities from each inner list
        extracted_entities = [[item[1], item[2]] for item in list_of_lists]
        return extracted_entities
    except Exception as e:
        print(f"Error processing row: {e}")
        return []

# Apply the function to each row in the DataFrame
df['extracted_entities'] = df.apply(extract_entities, axis=1)

# Print the updated DataFrame
print(df[['entities', 'extracted_entities']])


In [77]:
# Count the number of rows where 'extracted_entities' is an empty list
empty_count = df['extracted_entities'].apply(lambda x: len(x) != 0).sum()

print(f"Number of rows with empty lists: {empty_count}")


Number of rows with empty lists: 334


In [46]:

# Load ClinicalBERT tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")
model = AutoModel.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")

# Function to tokenize entities, extract features, and return the linear layer output
def extract_linear_layer_features(first_entity, second_entity):
    # Tokenize both entities
    first_entity_tokens = tokenizer(first_entity, padding=True, truncation=True, return_tensors="pt")
    second_entity_tokens = tokenizer(second_entity, padding=True, truncation=True, return_tensors="pt")
    
    # Feed the tokenized inputs into ClinicalBERT model
    with torch.no_grad():  # Disable gradient calculation for inference
        first_entity_output = model(**first_entity_tokens)
        second_entity_output = model(**second_entity_tokens)
    
    # Extract the last hidden states (features from the last layer)
    first_entity_features = first_entity_output.last_hidden_state.squeeze().mean(dim=0).tolist()
    second_entity_features = second_entity_output.last_hidden_state.squeeze().mean(dim=0).tolist()
    
    # Return the features (mean pooled features)
    return first_entity_features, second_entity_features

# Apply the function to each row in the DataFrame (using First Entity and Second Entity columns)
new_df[['first_entity_features', 'second_entity_features']] = new_df.apply(
    lambda row: pd.Series(extract_linear_layer_features(row['First Entity'], row['Second Entity'])), axis=1)

# Print the updated DataFrame with the extracted features
new_df[['first_entity_features', 'second_entity_features']]


/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/436M [00:00<?, ?B/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


,first_entity_features,second_entity_features
0,"[0.4430183172225952, 0.1934741586446762, -0.09...","[0.3027256429195404, 0.4631403386592865, -0.31..."
1,"[0.1683700829744339, 0.3361179530620575, -0.14...","[0.3027256429195404, 0.4631403386592865, -0.31..."
2,"[0.7711017727851868, -0.11166192591190338, -0....","[0.510710597038269, -0.0046751052141189575, 0...."
3,"[0.10064554214477539, 0.5451705455780029, -0.1...","[0.3453401029109955, 0.5212418437004089, -0.03..."
4,"[0.49189281463623047, 0.3442590832710266, -0.0...","[0.3453401029109955, 0.5212418437004089, -0.03..."
...,...,...
42471,"[0.7659021019935608, 0.35733509063720703, -0.5...","[0.3027256429195404, 0.4631403386592865, -0.31..."
42472,"[0.55926513671875, 0.7408992648124695, -0.1880...","[0.07500465214252472, 0.2318965494632721, 0.23..."
42473,"[0.3948667049407959, 0.4379931390285492, 0.172...","[0.24426205456256866, 0.2699955403804779, 0.20..."
42474,"[0.8447578549385071, 0.6879967451095581, -0.18...","[0.3453401029109955, 0.5212418437004089, -0.03..."


In [78]:
new_df


,First Entity,Second Entity,first_entity_token_ids,first_entity_attention_mask,second_entity_token_ids,second_entity_attention_mask,first_entity_features,second_entity_features,encoded_second_entity
0,NEUROPATHIC PAIN,PROBLEM,"[101, 24928, 11955, 21745, 2489, 102]","[1, 1, 1, 1, 1, 1]","[101, 2463, 102]","[1, 1, 1]","[0.4430183172225952, 0.1934741586446762, -0.09...","[0.3027256429195404, 0.4631403386592865, -0.31...",23
1,thoracotomy syndrome,PROBLEM,"[101, 24438, 6533, 18982, 18574, 9318, 102]","[1, 1, 1, 1, 1, 1, 1]","[101, 2463, 102]","[1, 1, 1]","[0.1683700829744339, 0.3361179530620575, -0.14...","[0.3027256429195404, 0.4631403386592865, -0.31...",23
2,spinal cord,ANATOMICAL_STRUCTURE,"[101, 19245, 13408, 102]","[1, 1, 1, 1]","[101, 1126, 10024, 7257, 1348, 168, 2401, 102]","[1, 1, 1, 1, 1, 1, 1, 1]","[0.7711017727851868, -0.11166192591190338, -0....","[0.510710597038269, -0.0046751052141189575, 0....",0
3,stimulator trial,PROCEDURE,"[101, 188, 3121, 13601, 13389, 3443, 102]","[1, 1, 1, 1, 1, 1, 1]","[101, 7791, 102]","[1, 1, 1]","[0.10064554214477539, 0.5451705455780029, -0.1...","[0.3453401029109955, 0.5212418437004089, -0.03...",24
4,epidural,PROCEDURE,"[101, 174, 25786, 12602, 102]","[1, 1, 1, 1, 1]","[101, 7791, 102]","[1, 1, 1]","[0.49189281463623047, 0.3442590832710266, -0.0...","[0.3453401029109955, 0.5212418437004089, -0.03...",24
...,...,...,...,...,...,...,...,...,...
42471,pain,PROBLEM,"[101, 2489, 102]","[1, 1, 1]","[101, 2463, 102]","[1, 1, 1]","[0.7659021019935608, 0.35733509063720703, -0.5...","[0.3027256429195404, 0.4631403386592865, -0.31...",23
42472,drain,MEDICAL_DEVICE,"[101, 13734, 102]","[1, 1, 1]","[101, 2657, 168, 4442, 102]","[1, 1, 1, 1, 1]","[0.55926513671875, 0.7408992648124695, -0.1880...","[0.07500465214252472, 0.2318965494632721, 0.23...",12
42473,STitle,MEDICINE,"[101, 188, 3121, 5034, 102]","[1, 1, 1, 1, 1]","[101, 5182, 102]","[1, 1, 1]","[0.3948667049407959, 0.4379931390285492, 0.172...","[0.24426205456256866, 0.2699955403804779, 0.20...",13
42474,drains removed,PROCEDURE,"[101, 20681, 2856, 102]","[1, 1, 1, 1]","[101, 7791, 102]","[1, 1, 1]","[0.8447578549385071, 0.6879967451095581, -0.18...","[0.3453401029109955, 0.5212418437004089, -0.03...",24


In [63]:

# Initialize the LabelEncoder
label_encoder = LabelEncoder()

# Fit and transform the "Second Entity" column
new_df['encoded_second_entity'] = label_encoder.fit_transform(new_df['Second Entity'])

# Create a dictionary that maps original labels to encoded values
label_dict = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))

# Print the dataframe with encoded labels
print(new_df[['Second Entity', 'encoded_second_entity']])

# Print the dictionary with labels and their corresponding encoded values
print("Label Dictionary:", label_dict)


              Second Entity  encoded_second_entity
0                   PROBLEM                     23
1                   PROBLEM                     23
2      ANATOMICAL_STRUCTURE                      0
3                 PROCEDURE                     24
4                 PROCEDURE                     24
...                     ...                    ...
42471               PROBLEM                     23
42472        MEDICAL_DEVICE                     12
42473              MEDICINE                     13
42474             PROCEDURE                     24
42475               PROBLEM                     23

[42476 rows x 2 columns]
Label Dictionary: {'ANATOMICAL_STRUCTURE': 0, 'BF_RESULT': 1, 'BM_RESULT': 2, 'BM_UNIT': 3, 'BM_VALUE': 4, 'BODY_FUNCTION': 5, 'BODY_MEASUREMENT': 6, 'FAMILY': 7, 'LABORATORY_DATA': 8, 'LAB_RESULT': 9, 'LAB_UNIT': 10, 'LAB_VALUE': 11, 'MEDICAL_DEVICE': 12, 'MEDICINE': 13, 'MED_DOSE': 14, 'MED_DURATION': 15, 'MED_FORM': 16, 'MED_FREQUENCY': 17, 'MED_ROUTE': 18,

In [ ]:
new_df

In [61]:
# Remove any trailing spaces from the column names if necessary
new_df.columns = new_df.columns.str.strip()

# Create the final DataFrame with the correct columns
final_df = pd.DataFrame({
    'first_entity_features': new_df['first_entity_features'],
    'encoded_second_entity': new_df['encoded_second_entity']
})

# Print the resulting DataFrame
print(final_df.head())


                               first_entity_features  encoded_second_entity
0  [0.4430183172225952, 0.1934741586446762, -0.09...                     23
1  [0.1683700829744339, 0.3361179530620575, -0.14...                     23
2  [0.7711017727851868, -0.11166192591190338, -0....                      0
3  [0.10064554214477539, 0.5451705455780029, -0.1...                     24
4  [0.49189281463623047, 0.3442590832710266, -0.0...                     24


In [62]:


# Assuming 'new_df' is your DataFrame
# Preprocessing the 'first_entity_features' column to make it usable for training
X = np.array(final_df['first_entity_features'].tolist())  # Convert list of features to numpy array
y = final_df['encoded_second_entity']  # Labels for classification

# Split the dataset into train and test sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Build the Classification Model
model = Sequential()

# Input layer and first hidden layer
model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))

# Second hidden layer
model.add(Dense(32, activation='relu'))

# Output layer (number of classes depends on the unique values in y)
model.add(Dense(len(np.unique(y)), activation='softmax'))

# Compile the model
model.compile(optimizer=Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

# Evaluate the model on the test data
loss, accuracy = model.evaluate(X_test, y_test)

print(f"Test Accuracy: {accuracy * 100:.2f}%")

# Predict on new data (example prediction on X_test)
predictions = model.predict(X_test)
print(f"Predictions on test data: {np.argmax(predictions, axis=1)}")


/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10


I0000 00:00:1736255653.354952     184 service.cc:145] XLA service 0x7d938c020160 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1736255653.354996     184 service.cc:153]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1736255653.355002     184 service.cc:153]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5


 118/1062 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.3672 - loss: 2.5103

I0000 00:00:1736255654.827215     184 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1062/1062 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.6728 - loss: 1.2315 - val_accuracy: 0.8358 - val_loss: 0.5265
Epoch 2/10
1062/1062 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8648 - loss: 0.4489 - val_accuracy: 0.8752 - val_loss: 0.4191
Epoch 3/10
1062/1062 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8777 - loss: 0.3934 - val_accuracy: 0.8804 - val_loss: 0.3869
Epoch 4/10
1062/1062 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8859 - loss: 0.3570 - val_accuracy: 0.8876 - val_loss: 0.3595
Epoch 5/10
1062/1062 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8919 - loss: 0.3418 - val_accuracy: 0.8862 - val_loss: 0.3790
Epoch 6/10
1062/1062 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8992 - loss: 0.3194 - val_accuracy: 0.8888 - val_loss: 0.3456
Epoch 7/10
1062/1062 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9024 - loss: 0.2983 - val_accuracy: 0.8911 - val_loss: 0.3432
Epoch 8/10
1062/1062 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9047 - loss: 0.2808 - val_accurac

In [72]:
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report, confusion_matrix  # Import the necessary metrics

predictions = model.predict(X_test)

# Convert predictions to class labels
predicted_labels = np.argmax(predictions, axis=1)

# Map encoded predictions back to original labels
predicted_labels_original = [key for pred in predicted_labels for key, value in label_dict.items() if value == pred]

# Print the mapped predictions
# print(f"Predicted Labels: {predicted_labels_original}")

# Additional: Evaluate performance using classification report and confusion matrix
print("Classification Report:")
print(classification_report(y_test, predicted_labels))  # Evaluate using classification report
print("Confusion Matrix:")
print(confusion_matrix(y_test, predicted_labels))  # Show confusion matrix

266/266 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
Classification Report:
              precision    recall  f1-score   support

           0       0.92      0.95      0.93       875
           1       0.75      0.48      0.58        82
           2       0.48      0.82      0.60        84
           3       0.77      0.91      0.84        79
           4       0.68      0.70      0.69       237
           5       0.99      0.84      0.91       161
           6       0.93      0.91      0.92       290
           7       1.00      1.00      1.00         9
           8       0.87      0.92      0.89       627
           9       0.70      0.66      0.68       125
          10       1.00      0.13      0.23        23
          11       0.84      0.78      0.81       403
          12       0.83      0.75      0.79       106
          13       0.90      0.91      0.91       931
          14       0.87      0.80      0.83       116
          15       1.00      0.77      0.87        30
          16     

/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [44]:
import pandas as pd
from transformers import AutoTokenizer

# Assuming df is already defined, or you can define it like this:
# df_token = df.copy()  # If you're working with an existing DataFrame df

# Load ClinicalBERT tokenizer (you can use BioBERT or other clinical models)
tokenizer = AutoTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")

# Function to tokenize entities and extract features
def tokenize_and_extract_features(entities):
    # Check if the list contains at least two entities
    if len(entities) >= 2:
        first_entity = entities[0]
        second_entity = entities[1]
        
        # Tokenize both entities
        first_entity_tokens = tokenizer(first_entity, padding=True, truncation=True, return_tensors="pt")
        second_entity_tokens = tokenizer(second_entity, padding=True, truncation=True, return_tensors="pt")
        
        # Extract token IDs and attention mask for both entities
        first_entity_token_ids = first_entity_tokens['input_ids'].squeeze().tolist()
        first_entity_attention_mask = first_entity_tokens['attention_mask'].squeeze().tolist()
        
        second_entity_token_ids = second_entity_tokens['input_ids'].squeeze().tolist()
        second_entity_attention_mask = second_entity_tokens['attention_mask'].squeeze().tolist()
        
        # Return the features as a tuple
        return first_entity_token_ids, first_entity_attention_mask, second_entity_token_ids, second_entity_attention_mask
    else:
        # If the list does not have two entities, return None or empty lists
        return [], [], [], []

# Apply the function to each row in the DataFrame
df_token[['first_entity_token_ids', 'first_entity_attention_mask', 'second_entity_token_ids', 'second_entity_attention_mask']] = new_df.apply(
    lambda x: pd.Series(tokenize_and_extract_features(x)))

# Print the updated DataFrame with token IDs and attention masks
print(df_token[['first_entity_token_ids', 'first_entity_attention_mask', 'second_entity_token_ids', 'second_entity_attention_mask']])


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


NameError: name 'df_token' is not defined